# NetBAT - Network Behavioral Analytics Tool

Rob Osterburg, Galvanize Data Science Immersive Project Proposal

### Motivation

Incident response is what happens after a security breach, and not surprisingly is a growing sector of the information security business.  Mandiant is a leading incident response company and has resolved many breaches including:  Equifax, the Clinton Campaign, and Target.  They report that **attackers in the U.S. go undetected for more than 3 months on average**.  During this time attackers need to maintain a presence inside the compromised network and to ship data out.  All this traffic must go through whatever protections the organization has in place.  My project focuses on detecting these signals before the objective of the attack is obtained.  

![Idealized attack against a retail target with malware rendered in red.](/Users/rob/Google_Drive/Datascience/Galvanize/Project/wakeful/images/retail_attack_malware_deployment.png)

MIT AI2 system is the for inspiration for my project.   

In the *The Innovators*, Walter Isaacson asks:

>"is it possible that humans and machines working in partnership will be indefinitely more powerful than an artificial intelligence machine working alone?” 

I believe the answer it **yes** and MIT is currently applying this idea to network security.  Their AI2 system uses unsupervised learning to make recommendations to an analyst who labels the events as either normal or attack.  A supervised learning algorithm then uses the labeled data to improve the selection of future analyst alerts.

[MIT AI2 with analyst empowered by machine learning](http://news.mit.edu/2016/ai-system-predicts-85-percent-cyber-attacks-using-input-human-experts-0418). 

![MIT AI^2 system](/Users/rob/Google_Drive/Datascience/Galvanize/Project/wakeful/images/overall_solution_context.png)


### Data
[Security Onion](https://securityonion.net) is a distribution of Linux focused on network monitoring.  Developed and maintained by the incident responders at Mandiant, SO includes tools to gather and analyze network traffic, and one in particular - [Bro Security Network Monitor (BSNM)](https://www.bro.org/) - is perfect for feature engineering.  BSNM understands network protocols and produces log files for each. 

I was unable to find a set of labeled data to use for this project.  So, I decided to gather data from my own home network, and use it to represent normal data.  After a couple of weeks, I now have ~40,000 DNS and ~80,000 connection log entries.  Beyond simply logging the traffic SO gives you means to investigate events using both beohavior-based ([BSNM](https://www.bro.org/)) and signature-based ([Snort](https://www.snort.org/)) detection tools.  Investigating my own network, I found one computer that appeared to have malware and have since wiped and re-imaged the system.  Now that my network appears to be free of malware, I think my plan to label its DNS traffic normal is reasonable.  

Eric Conrad is a [SANS instructor](https://www.sans.org/instructors/eric-conrad/date/desc/) and the CTO for a security company recently gave a talk on how malware communicates with its command and control (C2) server using DNS tunneling.  His [talk at Security Onion Con 2016](https://youtu.be/ViR405l-ggg) and his [related blog post](http://www.ericconrad.com/2016/09/c2-phone-home-leveraging-securityonion.html) includes links to BSNM logs for four different malicious uses of DNS including both tunneling and C2 communications and contain ~6,500 DNS and ~3,500 connection log entries.  I plan on using these data as the basis for my attack data.

I plan will have an EDA of these data sets completed by Monday morning Jan 8.  


#### Feature Engineering

* Why the focus on DNS: DNS answers the question of what IP address has been assigned to a URL.  By design, if a query can't answered locally it is forwarded to the root server for that top-level domain, and then recursively on down to an authoritative server.  DNS is an essential service for any organization and is rarely monitored. Even from deep within a organizations network most systems have DNS access and the forwarding behavior of this protocol enables DNS packets to reach the internet.  Just as the DNS response packets are let back in. This makes it a perfect communication channel for attackers.    

* Indicators of compromise -- Derived from the YouTube talks listed in the Citations Section.

    * DNS Protcol
    
        * Unusually long query strings
        
        * TXT, NULL and QUERY packets are used to transfer base-64 encoded data
        
        * NULL packets — used to transfer binary data
        
        * Large number of requests to hosts or subdomains
        
        * Length of time URL has been registered (some "fast flux" domains change the IPs they are associated with every ~150 seconds)
        
        * Rate of queries from a source IP address visits is much higher than average
        
        * False positives include Amazon URLs and others that use a hash as the subdomain
        
    * ICMP Protocol
    
        * Data portion differs from what the various OS ping implementations send
        
        * Packet size is large (i.e., greater than 200 or 400 bytes)
        
        * Rate is more rapid than once per second


### Minimum Viable Product

* Extract data from the BNSM DNS and connection logs

* Classify DNS packets as Normal, Attack and Uncertain 

* Assemble a set of reasonably representative data from the sources cited in the Data Section

* Select a supervised model for classifying events as normal, uncertain or attack.  Ideas: hierarchical model, random forest or gradient boosting

* Prefer a model whose results will be informative to non-data-scientists

#### MVP+

* Metric to quantify how much better the model performs in comparison to blacklist, whitelist or simple rule-based approaches
    
#### MVP++

* Develop a similar model for the ICMP protocol

### Deliverables
* Python code to process the logs and to model the data
* Repository with the code, tests, example data, findings and a presentation


### Business Value

Blacklisting and whitelisting are core practices to security and IT practitioners.  The idea goes back to firewalls which must either pass a packet or block it.  Anything on the whitelist is passed, while anything on the blacklist is blocked.  By evaluating my project in comparison to a blacklist/whitelist approach, I hope to make its results accessible to professionals in an industry where I hope to be hired.

How can we apply the blacklist / whitelist idea to DNS attack traffic?  If the security team at an organization uses a blacklist approach where they maintain a list of blocked URLs. In so doing, they give the attacker the advantage by allowing them to make small changes to domain names they use to avoid having their traffic from being blocked.  Whitelists on the other hand also disadvantage the security team because in addition to finding the malicious traffic, they also produce a lot of false positives.  Security teams tend to be lean because they are overhead expense to the organization that makes the impact of a whitelist approach is all the more.

I believe that machine learning approach based on protocol-specific behavior provides value over both the blacklist and whitelist approaches, here is why:

* Machine learning is better than the blacklist approach because it results in higher recall (i.e., fewer false negatives) by learning to detect and block similar malicious traffic with minimal human intervention.

* Machine learning is also better than the whitelist approach by higher precision (i.e., fewer false positives) by learning to detect similar normal traffic.    

### Citations
* [Security Onion 2016: C2 Phone Home - Eric Conrad](https://youtu.be/ViR405l-ggg)
* [Chris McCubbin,  Machine learning applied to Bro](https://youtu.be/ZV5Ckf9wLrc)
* [Data Analysis, Machine Learning, Bro, and You! by Brian Wylie](https://youtu.be/pG5lU9CLnIU)
* [BNSM DNS Log Documentation](https://www.bro.org/sphinx/scripts/base/protocols/dns/main.bro.html#type-DNS::Info)
* [BNSM ICMP Log Documentation](https://www.bro.org/sphinx/scripts/base/bif/plugins/Bro_ICMP.events.bif.bro.html)
* [BNSM Conn Log Documentation](https://www.bro.org/sphinx/scripts/base/protocols/conn/main.bro.html#type-Conn::Info)


#### DNS Log

![DNS Log Fields](/Users/rob/Google_Drive/Datascience/Galvanize/Project/wakeful/images/dns-log-fields.png)

![DNS Log Example](/Users/rob/Google_Drive/Datascience/Galvanize/Project/wakeful/images/dns-log-example.png)

#### Connection Log

![Conn Log Fields](/Users/rob/Google_Drive/Datascience/Galvanize/Project/wakeful/images/conn-log-fields.png)

![Conn Log Example](/Users/rob/Google_Drive/Datascience/Galvanize/Project/wakeful/images/conn-log-example.png)
 
Note: These logs can be joined using the connection ID.  
